In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
PATH = "/content/drive/MyDrive/WASSA"

In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dataset_folder = '/content/drive/MyDrive/WASSA/'
destination_folder = '/content/drive/MyDrive/WASSA'

In [ ]:
 # Load data and set labels
test = pd.read_csv(dataset_folder+'test.tsv',sep='\t')
#print(data_val.shape)
data_train = pd.read_csv(dataset_folder+'train.tsv',sep='\t')
data_val   = pd.read_csv(dataset_folder+'dev.tsv',sep='\t')
ground = pd.read_csv(dataset_folder+'/res/goldstandard.tsv',sep='\t',names=["empathy", "distress", 'emotion'])
#ground = pd.read_csv('/content/drive/MyDrive/WASSA/res/goldstandard.tsv')
# Display 5 random samples
data_val.head()

In [ ]:
train

In [ ]:
data_val['empathy'] = ground['empathy']
data_val['distress']= ground['distress']
data_val['emotion'] = ground['emotion']

data_train.sample()
train = data_train[['distress','empathy','essay', 'gender','education', 'race', 'age','income', 'personality_conscientiousness', 'personality_openess', 'personality_extraversion', 'personality_agreeableness', 'personality_stability', 'iri_perspective_taking', 'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern']]
val = data_val[['distress','empathy','essay', 'gender','education', 'race', 'age','income', 'personality_conscientiousness', 'personality_openess', 'personality_extraversion', 'personality_agreeableness', 'personality_stability', 'iri_perspective_taking', 'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern']]

In [ ]:
test

In [ ]:
X_train, X_val, y_train, y_val = train.essay.values,val.essay.values,train.empathy.values,val.empathy.values
print(X_train.shape)
#X_t = np.vstack((train.education.values,train.age.values,train.income.values)).T
#X_t = X_t.astype(np.float32)
#for i in range(3):
    # k = np.max(X_t[:,i])
    
    # print(k)
    # X_t[:,i] = X_t[:,i]/np.float(k)
#print(X_t.shape)
#X_v = np.vstack((val.education.values,val.age.values,val.income.values)).T
#X_v = X_v.astype(np.float32)
#for i in range(3):
    # k = np.max(X_v[:,i])
    # print(k)
    # X_v[:,i] = X_v[:,i]/np.float(k)
#print(X_v.shape)
#print(X_train.shape)

In [ ]:
print(X_train.shape, X_val.shape)

In [ ]:
#X_test,y_test = test.essay.values,test.empathy.values
#y_t = np.vstack((test.education.values,test.age.values,test.income.values)).T
#y_t = y_t.astype(np.float32)
#for i in range(3):
#    k = np.max(y_t[:,i])
#    # print(k)
#    y_t[:,i] = y_t[:,i]/np.float(k)
# print(y_t)


In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!pip3 install transformers sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModel

In [ ]:
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
# Print sentence 0
print('Original: ', X_train[0])
print('Processed: ', text_preprocessing(X_train[0]))

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
# Specify `MAX_LEN`
MAX_LEN = 256

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X_train[0]])[0].squeeze().numpy())
print('Original: ', X_val[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train).float()
val_labels = torch.tensor(y_val).float()
#X_t_labels = torch.tensor(X_t).float()
#X_v_labels = torch.tensor(X_v).float()
# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
#train_data = TensorDataset(train_inputs, train_masks, train_labels, X_t_labels)
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
#val_data = TensorDataset(val_inputs, val_masks, val_labels ,X_v_labels)
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
############### CREATING TEST_DATA_LOADER #######################
# Convert other data types to torch.Tensor
#test_inputs, test_masks = preprocessing_for_bert(X_test)
#test_labels = torch.tensor(y_test).float()
#y_t_labels = torch.tensor(y_t).float()
# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our test set
#test_data = TensorDataset(test_inputs, test_masks, test_labels,y_t_labels)
#test_sampler = RandomSampler(test_data)
#test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [ ]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 1

        # Instantiate BERT model
        self.bert = AutoModel.from_pretrained("bert-base-uncased")
        
        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(H, D_out),
            nn.ReLU()
        )
        
        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask, decoder_input_ids=None, decoder_attention_mask=None):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        # logits = self.classifier(torch.cat((last_hidden_state_cls,X_t),1))
        logits = self.classifier(last_hidden_state_cls)
        #regress = self.classifier2(X_t)
        #final = self.classifier3(torch.cat((logits,regress),1))
        return logits

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=2e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
import random
import time

# Specify loss function
loss_fn = nn.MSELoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    Best_acc = 0.0
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)
            logits = torch.squeeze(logits)
            #print(logits.shape," hello_in_train")
            
            #print(logits)
            #print(b_labels)
            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            if val_accuracy>Best_acc:
              Best_acc = val_accuracy
              torch.save(model, PATH+"/"+str(epoch_i)+"empathy_best_model.pt")
            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")

def pearson(X,Y):
    xm = torch.mean(X)
    ym = torch.mean(Y)
    num = 0.0
    deno1 = 0.0
    deno2 = 0.0
    deno = 0.0
    n = X.shape[0]
    for i in range(n):
        num += (X[i]-xm)*(Y[i]-ym)
        deno1 += (X[i]-xm)*(X[i]-xm)
        deno2 += (Y[i]-ym)*(Y[i]-ym)
    deno = deno1*deno2
    deno = torch.sqrt(deno)
    return (num/deno).item()

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask) 

        logits = torch.squeeze(logits)
        #print('here in evaluate',logits.shape)
        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())
        #print('loss ', val_loss)

        # Get the predictions
        preds = logits                                                    #######################

        # Calculate the accuracy rate
        # accuracy = (preds == b_labels).cpu().numpy().mean() * 100         #######################(pearson_Score)
        accuracy = pearson(preds,b_labels)
        #print('accuracy ',accuracy)
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
    print(val_loss," halo ",val_accuracy)

    return val_loss, val_accuracy

In [ ]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=10)
train(bert_classifier, train_dataloader, val_dataloader, epochs=10, evaluation=True)

In [ ]:
def custom_dataset(x,y):
  inputs, masks = preprocessing_for_bert(x)
  labels = torch.tensor(y).float()
  data = TensorDataset(inputs, masks, labels)
  #sampler = RandomSampler(data)
  dataloader = DataLoader(data, shuffle = False, batch_size=batch_size)
  return dataloader

def custom_data(val,train):
  X_val, y_val_distress, y_val_empathy = val.essay.values,train.distress.values,train.empathy.values
  ds_loader = custom_dataset(X_val,y_val_distress)
  em_loader = custom_dataset(X_val,y_val_empathy)
  return ds_loader , em_loader


In [ ]:
def eval(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()
    model.to(device)
    # Tracking variables
    val_accuracy = []
    val_loss = []

    results = torch.tensor([])
    labels = torch.tensor([])
    results = results.to(device)
    labels = labels.to(device)
    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask,None) 

        logits = torch.squeeze(logits)
        #print('here in evaluate',logits.shape)
        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())
        #print('loss ', val_loss)

        # Get the predictions
        preds = logits
        preds = preds.to(device)
        b_labels = b_labels.to(device)                                                    #######################
        results = torch.cat((results,preds),0)
        labels = torch.cat((labels,b_labels),0)
        # Calculate the accuracy rate
        # accuracy = (preds == b_labels).cpu().numpy().mean() * 100         #######################(pearson_Score)
        accuracy = pearson(preds,b_labels)
        #print('accuracy ',accuracy)
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.sum(val_accuracy)/len(val_accuracy)
    print(val_loss,"= loss , pearson = ",val_accuracy)

    return results,labels

In [ ]:
model_ds = torch.load('/content/drive/MyDrive/WASSA/8distress_best_model.pt')
model_em = torch.load('/content/drive/MyDrive/WASSA/2empathy_best_model.pt')
#len(test)

In [ ]:
ds,em = custom_data(train,train)

In [ ]:
train.shape

In [ ]:
def embds(data,model):
  tm = torch.tensor([])
  print(tm)
  tm.to('cpu')
  model_ds.eval()
  model_ds.to(device)
  for batch in data:
      # Load batch to GPU
      b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
      # print(b_labels)
      # Compute logits
      with torch.no_grad():
          outputs = model.bert(input_ids=b_input_ids,
                              attention_mask=b_attn_mask)
      ext = torch.tensor(outputs[0][:, 0, :]).to('cpu')
      tm = torch.cat((tm,ext),0)
  return(tm)                       

In [ ]:
ds_train,em_train = custom_data(train,train)
ds_val, em_val    = custom_data(val,val)
ds_test, em_test  = custom_data(test,train[0:len(test)])

In [ ]:
ds_tred, em_tred     = embds(ds_train,model_ds),embds(em_train,model_em) 
ds_valed, em_valed   = embds(ds_val,model_ds),embds(em_val,model_em)
ds_tested, em_tested = embds(ds_test,model_ds),embds(em_test,model_em)

In [ ]:
train_md = train.iloc[:,3:]
val_md   = val.iloc[:,3:]
test_md  = test.iloc[:,4:]

In [ ]:
cols1 = test_md.columns.tolist() 
cols2 = train_md.columns.tolist()
cols3 = val_md.columns.tolist()
print(cols1)
print(cols2)
print(cols3)

['gender', 'education', 'race', 'age', 'income', 'personality_conscientiousness', 'personality_openess', 'personality_extraversion', 'personality_agreeableness', 'personality_stability', 'iri_perspective_taking', 'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern']
['gender', 'education', 'race', 'age', 'income', 'personality_conscientiousness', 'personality_openess', 'personality_extraversion', 'personality_agreeableness', 'personality_stability', 'iri_perspective_taking', 'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern']
['gender', 'education', 'race', 'age', 'income', 'personality_conscientiousness', 'personality_openess', 'personality_extraversion', 'personality_agreeableness', 'personality_stability', 'iri_perspective_taking', 'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern']


In [ ]:
print(train_md.shape)
print(val_md.shape)
print(test_md.shape)
scaled(test_md)

In [ ]:
import pandas as pd
from sklearn import preprocessing

def scaled(other):
  x = other.values #returns a numpy array
  min_max_scaler = preprocessing.StandardScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  df = pd.DataFrame(x_scaled)
  return df

In [ ]:
ds_tred = pd.concat([pd.DataFrame(ds_tred.numpy()),scaled(train_md)],axis = 1)
em_tred = pd.concat([pd.DataFrame(em_tred.numpy()),scaled(train_md)],axis = 1)
ds_valed = pd.concat([pd.DataFrame(ds_valed.numpy()),scaled(val_md)],axis = 1)
em_valed = pd.concat([pd.DataFrame(em_valed.numpy()),scaled(val_md)],axis = 1)
ds_tested = pd.concat([pd.DataFrame(ds_tested.numpy()),scaled(test_md)],axis = 1)
em_tested = pd.concat([pd.DataFrame(em_tested.numpy()),scaled(test_md)],axis = 1)

**SVR**

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
def SVM(dx_train,dy_train,dx_val,dy_val,em_train,ey_train,ex_val,ey_val):
  rgr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
  rgr.fit(dx_train,dy_train)
  y_val = rgr.predict(dx_val)
  print('distress=',pearsonr(y_val,dy_val))
  rgr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
  rgr.fit(em_train,ey_train)
  y_val = rgr.predict(ex_val)
  print('empathy=',pearsonr(y_val,ey_val))

In [ ]:
SVM(tred.values,train.distress.values,valed.values,val.distress.values
       ,tred.values,train.empathy.values,valed.values,val.empathy.values)

In [ ]:
SVM(ds_tred.values,train.distress.values,ds_valed.values,val.distress.values
       ,em_tred.values,train.empathy.values,em_valed.values,val.empathy.values)

**LINEAR_REGRESSION**

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.datasets import make_regression
def metric(dx_train,dy_train,dx_val,dy_val,em_train,ey_train,ex_val,ey_val):
  rgr = ElasticNet(random_state=0)
  rgr.fit(dx_train,dy_train)
  y_val = rgr.predict(dx_val)
  print('distress=',pearsonr(y_val,dy_val))
  rgr = ElasticNet(random_state=0)
  rgr.fit(em_train,ey_train)
  y_val = rgr.predict(ex_val)
  print('empathy=',pearsonr(y_val,ey_val))
  rgr = Lasso(random_state=0)
  rgr.fit(dx_train,dy_train)
  y_val = rgr.predict(dx_val)
  print('distress_Lasso=',pearsonr(y_val,dy_val))
  rgr = Lasso(random_state=0)
  rgr.fit(em_train,ey_train)
  y_val = rgr.predict(ex_val)
  print('empathy_Lasso=',pearsonr(y_val,ey_val))

In [ ]:
metric(tred.values,train.distress.values,valed.values,val.distress.values
       ,tred.values,train.empathy.values,valed.values,val.empathy.values)

In [ ]:
metric(ds_tred.values,train.distress.values,ds_valed.values,val.distress.values
       ,em_tred.values,train.empathy.values,em_valed.values,val.empathy.values)

**Random-Forest**

In [ ]:
# Fitting Random Forest Regression to the dataset 
# import the regressor 
from sklearn.ensemble import RandomForestRegressor 

def RRF(dx_train,dy_train,dx_val,dy_val,em_train,ey_train,ex_val,ey_val):
  rgr = RandomForestRegressor(n_estimators = 100, random_state = 0)
  rgr.fit(dx_train,dy_train)
  y_val = rgr.predict(dx_val)
  print('distress=',pearsonr(y_val,dy_val))
  rgr = RandomForestRegressor(n_estimators = 100, random_state = 0)
  rgr.fit(em_train,ey_train)
  y_val = rgr.predict(ex_val)
  print('empathy=',pearsonr(y_val,ey_val))

In [ ]:
RRF(tred.values,train.distress.values,valed.values,val.distress.values
       ,tred.values,train.empathy.values,valed.values,val.empathy.values)

In [ ]:
RRF(ds_tred.values,train.distress.values,ds_valed.values,val.distress.values
       ,em_tred.values,train.empathy.values,em_valed.values,val.empathy.values)

**XG-BOOST**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xg 
def XGBOOST(dx_train,dy_train,dx_val,dy_val,em_train,ey_train,ex_val,ey_val):
  rgr = xg.XGBRegressor(objective ='reg:squarederror', 
                  n_estimators = 10, seed = 123)
  rgr.fit(dx_train,dy_train)
  y_val = rgr.predict(dx_val)
  print('distress=',pearsonr(y_val,dy_val))
  rgr = xg.XGBRegressor(objective ='reg:squarederror', 
                  n_estimators = 10, seed = 123)
  rgr.fit(em_train,ey_train)
  y_val = rgr.predict(ex_val)
  print('empathy=',pearsonr(y_val,ey_val))

In [ ]:
XGBOO(tred.values,train.distress.values,valed.values,val.distress.values
       ,tred.values,train.empathy.values,valed.values,val.empathy.values)

In [ ]:
XGBOOST(ds_tred.values,train.distress.values,ds_valed.values,val.distress.values
       ,em_tred.values,train.empathy.values,em_valed.values,val.empathy.values)

**ADA-BOOST**

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
def ADABOOST(dx_train,dy_train,dx_val,dy_val,em_train,ey_train,ex_val,ey_val):
  rgr = AdaBoostRegressor(learning_rate = 0.9,loss='square',random_state=0, n_estimators=100)
  rgr.fit(dx_train,dy_train)
  y_val = rgr.predict(dx_val)
  print('distress=',pearsonr(y_val,dy_val))
  rgr = AdaBoostRegressor(loss='square',random_state=0, n_estimators=100)
  rgr.fit(em_train,ey_train)
  y_val = rgr.predict(ex_val)
  print('empathy=',pearsonr(y_val,ey_val))

In [ ]:
ADABOOST(ds_tred.values,train.distress.values,ds_valed.values,val.distress.values
       ,em_tred.values,train.empathy.values,em_valed.values,val.empathy.values)

**MLP**

In [ ]:
PATH = '/content/drive/MyDrive/WASSA/res'

In [ ]:
from sklearn.neural_network import MLPRegressor
def MLP(dx_train,dy_train,dx_val,dy_val,em_train,ey_train,ex_val,ey_val,test_x):
  rgr = MLPRegressor(hidden_layer_sizes=(100,),learning_rate='adaptive',activation='relu',random_state=1, max_iter=80)
  rgr.fit(dx_train,dy_train)
  y_val = rgr.predict(dx_val)
  ds_pred = rgr.predict(test_x)
  print('distress=',pearsonr(y_val,dy_val))
  rgr = MLPRegressor(hidden_layer_sizes=(100,),learning_rate='adaptive',activation='relu',random_state=1, max_iter=40)
  rgr.fit(em_train,ey_train)
  y_val = rgr.predict(ex_val)
  print('empathy=',pearsonr(y_val,ey_val))
  em_pred = rgr.predict(test_x)
  test_sub = pd.DataFrame({'em':em_pred,'ds':ds_pred})
  test_ans = test_sub.to_csv("predictions_EMP.tsv",sep='\t',index=False,header=None)

In [ ]:
MLP(tred.values,train.distress.values,valed.values,val.distress.values
       ,tred.values,train.empathy.values,valed.values,val.empathy.values,tested.values)

In [ ]:
MLP(ds_tred.values,train.distress.values,ds_valed.values,val.distress.values
       ,em_tred.values,train.empathy.values,em_valed.values,val.empathy.values,ds_tested.values)

In [ ]:
MLP(ds_tred.values,train.distress.values,ds_valed.values,val.distress.values
       ,em_tred.values,train.empathy.values,em_valed.values,val.empathy.values)

In [ ]:
def pearsonr(x, y):
	"""
	Calculates a Pearson correlation coefficient. 
	"""

	assert len(x) == len(y), 'Prediction and gold standard does not have the same length...'

	xm = sum(x)/len(x)
	ym = sum(y)/len(y)

	xn = [k-xm for k in x]
	yn = [k-ym for k in y]

	r = 0 
	r_den_x = 0
	r_den_y = 0
	for xn_val, yn_val in zip(xn, yn):
		r += xn_val*yn_val
		r_den_x += xn_val*xn_val
		r_den_y += yn_val*yn_val

	r_den = np.sqrt(r_den_x*r_den_y)

	if r_den:
		r = r / r_den
	else:
		r = 0

	# Presumably, if abs(r) > 1, then it is only some small artifact of floating
	# point arithmetic.
	r = max(min(r, 1.0), -1.0)

	return round(r,4)

In [ ]:
results_ds,ds_labels = eval(model_ds,ds)
results_em,em_labels = eval(model_em,em)

In [ ]:
results_ds = results_ds.to('cpu').numpy()
ds_labels  = ds_labels.to('cpu').numpy()
results_em = results_em.to('cpu').numpy()
em_labels  = em_labels.to('cpu').numpy() 

In [ ]:
print(pearsonr(results_ds,ds_labels))
print(pearsonr(results_em,em_labels))

In [ ]:

pearson(results_em,em_labels)

In [ ]:
ds_pred = results_ds 
em_pred = results_em

In [ ]:
test_sub = pd.DataFrame({'em':em_pred,'ds':ds_pred})
test_ans = test_sub.to_csv(PATH+"/predictions_EMP.tsv",sep='\t',index=False,header=None)

In [ ]:
from __future__ import print_function
import sys
import os
from math import sqrt

to_round = 4



def readFileToList(filename):
	#eprint("Reading data from",filename)
	lines=filename.readlines()
	result=[]
	for x in lines:
		result.append(x.rstrip().split('\t'))
	filename.close()
	return result

def calculatePRF(gold,prediction):
	"""
	gold/prediction list of list of emo predictions 
	"""
	# initialize counters
	labels = set(gold+prediction)
	tp = dict.fromkeys(labels, 0.0)
	fp = dict.fromkeys(labels, 0.0)
	fn = dict.fromkeys(labels, 0.0)
	precision = dict.fromkeys(labels, 0.0)
	recall = dict.fromkeys(labels, 0.0)
	f = dict.fromkeys(labels, 0.0)
	# check every element
	for g,p in zip(gold,prediction):
		# TP 
		if (g == p):
			tp[g] += 1
		else:
			fp[p] += 1
			fn[g] += 1
   # print("Label\tTP\tFP\tFN\tP\tR\tF")
	for label in labels:
		recall[label] = 0.0 if (tp[label]+fn[label]) == 0.0 else (tp[label])/(tp[label]+fn[label])
		precision[label] = 1.0 if (tp[label]+fp[label]) == 0.0 else (tp[label])/(tp[label]+fp[label])
		f[label] = 0.0 if (precision[label]+recall[label])==0 else (2*precision[label]*recall[label])/(precision[label]+recall[label])
		microrecall = (sum(tp.values()))/(sum(tp.values())+sum(fn.values()))
		microprecision = (sum(tp.values()))/(sum(tp.values())+sum(fp.values()))
		microf = 0.0 if (microprecision+microrecall)==0 else (2*microprecision*microrecall)/(microprecision+microrecall)
	# Macro average
	macrorecall = sum(recall.values())/len(recall)
	macroprecision = sum(precision.values())/len(precision)
	macroF = sum(f.values())/len(f)

	accuracy = 0
	for label in labels:
		accuracy += tp[label]

	accuracy = accuracy/len(gold)

	return round(microrecall,to_round),round(microprecision,to_round),round(microf,to_round),round(macrorecall,to_round),round(macroprecision,to_round),round(macroF,to_round),round(accuracy,to_round)

def pearsonr(x, y):
	"""
	Calculates a Pearson correlation coefficient. 
	"""

	assert len(x) == len(y), 'Prediction and gold standard does not have the same length...'

	xm = sum(x)/len(x)
	ym = sum(y)/len(y)

	xn = [k-xm for k in x]
	yn = [k-ym for k in y]

	r = 0 
	r_den_x = 0
	r_den_y = 0
	for xn_val, yn_val in zip(xn, yn):
		r += xn_val*yn_val
		r_den_x += xn_val*xn_val
		r_den_y += yn_val*yn_val

	r_den = sqrt(r_den_x*r_den_y)

	if r_den:
		r = r / r_den
	else:
		r = 0

	# Presumably, if abs(r) > 1, then it is only some small artifact of floating
	# point arithmetic.
	r = max(min(r, 1.0), -1.0)

	return round(r,to_round)

def calculate_pearson(gold, prediction):
	"""
	gold/prediction are a list of lists [ emp pred , distress pred ]
	"""

	# converting to float
	gold = [float(k) for k in gold]
	prediction = [float(k) for k in prediction]

	return pearsonr(gold, prediction)

def calculate_metrics(golds, predictions, task1, task2):
	"""
	gold/prediction list of list of values : [ emp pred , distress pred , emo pred ]
	"""
	if task1:
		gold_empathy = [k[0] for k in golds]
		prediction_empathy = [k[0] for k in predictions]
		pearson_empathy = calculate_pearson(gold_empathy, prediction_empathy)

		gold_distress = [k[1] for k in golds]
		prediction_distress = [k[1] for k in predictions]
		pearson_distress = calculate_pearson(gold_distress, prediction_distress)
		avg_pearson = (pearson_empathy + pearson_distress)/2
	else:
		avg_pearson, pearson_empathy, pearson_distress = 0,0,0

	if task2:
		gold_emo = [k[2] for k in golds]
		prediction_emo = [k[2] for k in predictions]

		microrecall,microprecision,microf,macrorecall,macroprecision,macroF,accuracy = calculatePRF(gold_emo, prediction_emo)
	else:
		microrecall,microprecision,microf,macrorecall,macroprecision,macroF,accuracy = 0,0,0,0,0,0,0

	return avg_pearson, pearson_empathy, pearson_distress, microrecall, microprecision, microf, macrorecall, macroprecision, macroF, accuracy

nb_labels_EMP = 2
nb_labels_EMO = 1

def score(input_dir, output_dir):
	# unzipped submission data is always in the 'res' subdirectory
	submission_path = os.path.join(input_dir, 'res', 'predictions_EMP.tsv')
	if not os.path.exists(submission_path):
		print('Could not find submission file {0}'.format(submission_path))
		task1 = False
	else:
		submission_file = open(os.path.join(submission_path))
		# The 2 first columns
		predictedList_EMP = [k[:nb_labels_EMP] for k in readFileToList(submission_file)]
		task1 = True

	# unzipped submission data is always in the 'res' subdirectory
	submission_path = os.path.join(input_dir, 'res', 'predictions_EMO.tsv')
	if not os.path.exists(submission_path):
		print('Could not find submission file {0}'.format(submission_path))
		predictedList = predictedList_EMP
		task2 = False
	else:
		submission_file = open(os.path.join(submission_path))
		predictedList_EMO = [k[:nb_labels_EMO] for k in readFileToList(submission_file)]

		if task1:
			# concatening the lists of preds
			predictedList = [i+j for i,j in zip(predictedList_EMP, predictedList_EMO)]
		else:
			predictedList = [[0,0]+j for j in predictedList_EMO]
		task2 = True

	# unzipped reference data is always in the 'res' subdirectory
	truth_file = open(os.path.join(input_dir, 'res', 'predictions_EMP.tsv'))
	goldList = readFileToList(truth_file)
	if (len(goldList) != len(predictedList)):
		print("Number of labels is not aligned!")
		sys.exit(1)

	avg_pearson, pearson_empathy, pearson_distress, micror, microp, microf, macror, macrop, macrof, accuracy = calculate_metrics(goldList,predictedList, task1, task2)

	with open(os.path.join(output_dir, 'scores.txt'), 'w') as output_file:
		str_to_write = ''
		# Not sure of that. Useful if the participant want to do only one subtask. Need to see if the leaderboard of the subtask does not update if there are nothing on score.txt 
		if task1:
			str_to_write += "Empathy Pearson Correlation: {0}\nDistress Pearson Correlation: {1}\nAveraged Pearson Correlations: {2}\n".format(pearson_empathy, pearson_distress, avg_pearson)
		if task2:
			str_to_write += "Micro Recall: {0}\nMicro Precision: {1}\nMicro F1-Score: {2}\nMacro Recall: {3}\nMacro Precision: {4}\nMacro F1-Score: {5}\nAccuracy: {6}\n".format(micror,microp,microf,macror,macrop,macrof,accuracy) 
		output_file.write(str_to_write)

def main():
  input_dir = "/content/drive/MyDrive/WASSA"
  output_dir = "/content/drive/MyDrive/WASSA"
  score(input_dir, output_dir)
main()


In [ ]:
nm = PATH+'/scores.txt'